In [1]:
!unzip /content/VLSP2021.zip -d /content/VLSP2021

Archive:  /content/VLSP2021.zip
   creating: /content/VLSP2021/Dev-Muc/
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0001.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0002.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0003.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0004.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0005.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0006.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0007.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0008.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0009.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0010.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0011.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0012.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0013.muc  
  inflating: /content/VLSP2021/Dev-Muc/0_dev_doisong_0014.muc  
  inflating: /content/VLSP2021/D

In [17]:
!unzip /content/Testing.zip -d /content/Testing

Archive:  /content/Testing.zip
   creating: /content/Testing/Test/
  inflating: /content/Testing/Test/0_test_vnexpress_congnghe_0001.muc  
  inflating: /content/Testing/Test/0_test_vnexpress_congnghe_0002.muc  
  inflating: /content/Testing/Test/0_test_vnexpress_congnghe_0003.muc  
  inflating: /content/Testing/Test/0_test_vnexpress_congnghe_0004.muc  
  inflating: /content/Testing/Test/0_test_vnexpress_congnghe_0005.muc  
  inflating: /content/Testing/Test/0_test_vnexpress_congnghe_0006.muc  
  inflating: /content/Testing/Test/0_test_vnexpress_congnghe_0007.muc  
  inflating: /content/Testing/Test/0_test_vnexpress_congnghe_0008.muc  
  inflating: /content/Testing/Test/0_test_vnexpress_congnghe_0009.muc  
  inflating: /content/Testing/Test/0_test_vnexpress_congnghe_0010.muc  
  inflating: /content/Testing/Test/0_test_vnexpress_giaoduc_0001.muc  
  inflating: /content/Testing/Test/0_test_vnexpress_giaoduc_0002.muc  
  inflating: /content/Testing/Test/0_test_vnexpress_giaoduc_0003.muc  


In [18]:
import os
import re

def process_text(text):
    """
    Xử lý nội dung văn bản để phân tách và gán các cột theo định dạng token POS NER (BIO).
    """
    tokens = re.split(r'(\s+|<ENAMEX.*?>|</ENAMEX>)', text)
    tag_stack = []  # Stack lưu trữ các nhãn hiện tại
    result = []

    for token in tokens:
        token = token.strip()
        if not token:  # Bỏ qua token trống
            continue

        if token.startswith('<ENAMEX TYPE='):
            # Bắt đầu thẻ ENAMEX -> Lấy nhãn và đẩy vào stack
            tag = re.search(r'TYPE="([^"]+)"', token).group(1)
            tag_stack.append(tag)
        elif token == '</ENAMEX>':
            # Kết thúc thẻ ENAMEX -> Xóa nhãn khỏi stack
            if tag_stack:
                tag_stack.pop()
        else:
            # Tạo dòng dữ liệu với token và nhãn
            if tag_stack:
                current_tag = f'B-{tag_stack[-1]}' if len(tag_stack) == 1 else f'I-{tag_stack[-1]}'
            else:
                current_tag = 'O'
            result.append([token, current_tag])  # POS tag mặc định là 'O'

    return result

def process_folder(input_dir, output_file):
    """
    Xử lý tất cả file .muc trong thư mục và ghi kết quả vào file .txt.
    """
    with open(output_file, 'w', encoding='utf-8') as output:
        for filename in os.listdir(input_dir):
            if filename.endswith('.muc'):
                input_path = os.path.join(input_dir, filename)
                with open(input_path, 'r', encoding='utf-8') as file:
                    text = file.read()

                # Xử lý văn bản thành định dạng token-level
                processed_data = process_text(text)

                # Ghi từng token vào file output
                for token, ner in processed_data:
                    output.write(f"{token} {ner}\n")
                output.write("\n")  # Dòng trống giữa các file

if __name__ == "__main__":
    # Định nghĩa các thư mục input và file output
    input_output_mapping = {
        "/content/VLSP2021/Dev-Muc": "dev-muc.txt",
        "/content/VLSP2021/Train-Muc": "train.txt",
        "/content/Testing/Test": "tests.txt"
    }

    # Xử lý từng thư mục
    for input_dir, output_file in input_output_mapping.items():
        print(f"Processing {input_dir} -> {output_file}")
        process_folder(input_dir, output_file)

    print("Hoàn thành xử lý tất cả các thư mục!")


Processing /content/VLSP2021/Dev-Muc -> dev-muc.txt
Processing /content/VLSP2021/Train-Muc -> train.txt
Processing /content/Testing/Test -> tests.txt
Hoàn thành xử lý tất cả các thư mục!


In [3]:
import re
from collections import defaultdict, Counter
from tqdm import tqdm

In [4]:
# Load train.txt để học tham số
def load_data(file_path):
    sentences, labels = [], []
    current_sentence, current_labels = [], []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                token, label = line.strip().split()
                current_sentence.append(token)
                current_labels.append(label)
            else:
                sentences.append(current_sentence)
                labels.append(current_labels)
                current_sentence, current_labels = [], []

    return sentences, labels

In [5]:
from tqdm import tqdm  # Thư viện để hiển thị tiến trình
from collections import defaultdict, Counter

def train_hmm(sentences, labels):
    """
    Học tham số HMM dựa trên token và NER tag.

    Args:
        sentences: Danh sách các câu, mỗi câu là một danh sách các token.
        labels: Danh sách các nhãn NER tương ứng với các câu.

    Returns:
        Tuple: Gồm các tham số của mô hình HMM:
            - initial_probs: Xác suất khởi đầu của các trạng thái.
            - transition_probs: Xác suất chuyển đổi giữa các trạng thái.
            - emission_probs: Xác suất phát xạ của các token từ các trạng thái.
            - states: Danh sách tất cả các trạng thái NER.
    """
    initial_probs = Counter()
    transition_probs = defaultdict(Counter)
    emission_probs = defaultdict(Counter)
    total_states = Counter()

    print("Bắt đầu huấn luyện mô hình HMM...")
    for sentence, label_seq in tqdm(zip(sentences, labels), total=len(sentences), desc="Đang huấn luyện"):
        initial_probs[label_seq[0]] += 1  # Đếm trạng thái khởi đầu
        for i in range(len(label_seq)):
            total_states[label_seq[i]] += 1  # Đếm số lần xuất hiện của mỗi trạng thái
            emission_probs[label_seq[i]][sentence[i]] += 1  # Đếm số lần token được phát xạ từ trạng thái
            if i > 0:
                transition_probs[label_seq[i - 1]][label_seq[i]] += 1  # Đếm số lần chuyển đổi giữa các trạng thái

    # Chuẩn hóa xác suất
    initial_probs = {k: v / sum(initial_probs.values()) for k, v in initial_probs.items()}
    transition_probs = {k: {k2: v2 / sum(v.values()) for k2, v2 in v.items()} for k, v in transition_probs.items()}
    emission_probs = {k: {k2: v2 / sum(v.values()) for k2, v2 in v.items()} for k, v in emission_probs.items()}

    print("Huấn luyện xong!")
    return initial_probs, transition_probs, emission_probs, list(total_states.keys())

In [6]:
def viterbi(sentence, initial_probs, transition_probs, emission_probs, states):
    """
    Thuật toán Viterbi để tìm chuỗi nhãn NER có xác suất cao nhất cho một câu.

    Args:
        sentence: Danh sách các token của câu cần dự đoán.
        initial_probs: Xác suất khởi đầu của các trạng thái.
        transition_probs: Xác suất chuyển đổi giữa các trạng thái.
        emission_probs: Xác suất phát xạ của các token từ các trạng thái.
        states: Danh sách tất cả các trạng thái NER.

    Returns:
        Danh sách các nhãn NER dự đoán cho câu.
    """
    dp = [{} for _ in range(len(sentence))]  # Khởi tạo bảng DP
    path = [{} for _ in range(len(sentence))]  # Khởi tạo bảng lưu trữ đường đi

    # Bước khởi tạo: Xác suất cho trạng thái đầu tiên
    for state in states:
        dp[0][state] = initial_probs.get(state, 0) * emission_probs[state].get(sentence[0], 1e-6)
        path[0][state] = [state]  # Lưu đường đi

    # Bước quy hoạch động: Tính xác suất cho các trạng thái tiếp theo
    for t in range(1, len(sentence)):
        for curr_state in states:
            max_prob, prev_state = max(
                (dp[t - 1][prev_state] * transition_probs[prev_state].get(curr_state, 1e-6) *
                 emission_probs[curr_state].get(sentence[t], 1e-6), prev_state)  # Tính xác suất chuyển đổi và phát xạ
                for prev_state in states
            )
            dp[t][curr_state] = max_prob  # Lưu xác suất cao nhất
            path[t][curr_state] = path[t - 1][prev_state] + [curr_state]  # Lưu đường đi

    # Tìm đường đi tốt nhất: Trạng thái cuối cùng có xác suất cao nhất
    final_state = max(dp[-1], key=dp[-1].get)

    # Trả về đường đi tốt nhất (chuỗi nhãn NER)
    return path[-1][final_state]

In [7]:
from sklearn.metrics import classification_report
from tqdm import tqdm

def evaluate(dev_sentences, dev_labels, hmm_params):
    """
    Đánh giá hiệu suất của mô hình HMM trên tập development.

    Args:
        dev_sentences: Danh sách các câu trong tập development.
        dev_labels: Danh sách các nhãn NER tương ứng với các câu trong tập development.
        hmm_params: Các tham số của mô hình HMM (initial_probs, transition_probs, emission_probs, states).

    Returns:
        None. In ra kết quả đánh giá (precision, recall, F1-score).
    """
    initial_probs, transition_probs, emission_probs, states = hmm_params
    predictions = []

    print("\nĐang đánh giá mô hình HMM trên tập kiểm thử...")

    # Dự đoán nhãn cho các câu trong tập development
    for sentence in tqdm(dev_sentences, desc="Đang dự đoán"):
        pred = viterbi(sentence, initial_probs, transition_probs, emission_probs, states)
        predictions.append(pred)

    # Tính toán precision, recall, F1-scoreT
    y_true = [label for sublist in dev_labels for label in sublist]  # Nối tất cả các nhãn trong dev_labels
    y_pred = [label for sublist in predictions for label in sublist]  # Nối tất cả các nhãn trong predictions

    # Sử dụng classification_report để tính toán và in ra kết quả đánh giá
    print(classification_report(y_true, y_pred, zero_division=0))

In [8]:
# Main
train_file = '/content/train.txt'
dev_file = '/content/dev-muc.txt'
test_file = '/content/tests.txt'

In [9]:
train_sentences, train_labels = load_data(train_file)
dev_sentences, dev_labels = load_data(dev_file)

In [10]:
type(train_sentences)

list

In [11]:
train_sentences[0]

['Sau',
 "'Sống",
 'chung',
 'với',
 'mẹ',
 'chồng',
 "'",
 ',',
 'phim',
 "'Ngược",
 'chiều',
 'nước',
 'mắt',
 "'",
 'hứa',
 'hẹn',
 'gây',
 'bão',
 'với',
 'lý',
 'do',
 'này',
 'Sau',
 'thành',
 'công',
 'của',
 'phim',
 "'Sống",
 'chung',
 'với',
 'mẹ',
 'chồng',
 "'",
 ',',
 'phim',
 'truyền',
 'hình',
 "'Ngược",
 'chiều',
 'nước',
 'mắt',
 "'",
 'đang',
 'gây',
 'được',
 'sự',
 'chú',
 'ý',
 'với',
 'dàn',
 'diễn',
 'viên',
 'tuyệt',
 'đẹp',
 'và',
 'nội',
 'dung',
 'hấp',
 'dẫn',
 '.',
 'Chia',
 'sẻSau',
 'thành',
 'công',
 'của',
 'phim',
 "'Sống",
 'chung',
 'với',
 'mẹ',
 'chồng',
 "'",
 ',',
 'phim',
 'truyền',
 'hình',
 "'Ngược",
 'chiều',
 'nước',
 'mắt',
 "'",
 'đang',
 'gây',
 'được',
 'sự',
 'chú',
 'ý',
 'với',
 'dàn',
 'diễn',
 'viên',
 'tuyệt',
 'đẹp',
 'và',
 'nội',
 'dung',
 'hấp',
 'dẫn',
 '.',
 'Phim',
 'truyền',
 'hình',
 'Việt',
 'Nam',
 'phản',
 'ánh',
 'những',
 'câu',
 'chuyện',
 'đời',
 'thường',
 'về',
 'đề',
 'tài',
 'gia',
 'đình',
 'đang',
 'chiếm',
 '

In [12]:
print(len(train_sentences))

1050


In [13]:
train_sentences[1]

['Người',
 'phụ',
 'nữ',
 'bị',
 'xe',
 'tải',
 'cán',
 'gãy',
 'chân',
 ',',
 'cuốn',
 'xe',
 'máy',
 'vào',
 'gầm',
 'Đang',
 'lưu',
 'thông',
 'phía',
 'trước',
 ',',
 'chiếc',
 'xe',
 'máy',
 'bị',
 'ô',
 'tô',
 'tải',
 'đâm',
 'từ',
 'phía',
 'sau',
 'rồi',
 'cuốn',
 'vào',
 'gầm',
 '.',
 'Người',
 'phụ',
 'nữ',
 'điều',
 'khiển',
 'xe',
 'máy',
 'thoát',
 'chết',
 'trong',
 'gang',
 'tấc',
 '.',
 'Vụ',
 'tai',
 'nạn',
 'xảy',
 'ra',
 'vào',
 'khoảng',
 '16h30',
 'phút',
 'chiều',
 '22/9',
 'trên',
 'Quốc',
 'lộ',
 '1A',
 ',',
 'đoạn',
 'qua',
 'khối',
 'Tân',
 'Sơn',
 ',',
 'phường',
 'Quỳnh',
 'Thiện',
 ',',
 'TX',
 'Hoàng',
 'Mai',
 ',',
 'Nghệ',
 'An',
 '.',
 'Chiếc',
 'xe',
 'máy',
 'nát',
 'bét',
 'dưới',
 'bánh',
 'xe',
 'tải',
 '.',
 'Ảnh',
 ':',
 'Nguyễn',
 'Phê',
 'Vào',
 'thời',
 'gian',
 'trên',
 ',',
 'chị',
 'Hồ',
 'Thị',
 'Thúy',
 '(',
 'trú',
 'khối',
 '8',
 ',',
 'phường',
 'Quỳnh',
 'Thiện',
 ')',
 'điều',
 'khiển',
 'xe',
 'máy',
 'đi',
 'hướng',
 'Hoàng',
 'Mai

In [14]:
hmm_params = train_hmm(train_sentences, train_labels)

Bắt đầu huấn luyện mô hình HMM...


Đang huấn luyện: 100%|██████████| 1050/1050 [00:00<00:00, 1279.10it/s]


Huấn luyện xong!


In [15]:
import os

def count_files_in_directory(directory_path):
  """
  Đếm số lượng file trong một thư mục cụ thể.

  Args:
    directory_path: Đường dẫn đến thư mục cần đếm file.

  Returns:
    Số lượng file trong thư mục.
  """
  file_count = 0
  for filename in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, filename)):
      file_count += 1
  return file_count

# Đường dẫn đến thư mục bạn muốn đếm file
directory_path = "/content/VLSP2021/Train-Muc"

# Đếm số lượng file trong thư mục
number_of_files = count_files_in_directory(directory_path)

# In ra kết quả
print(f"Số lượng file trong thư mục {directory_path}: {number_of_files}")

Số lượng file trong thư mục /content/VLSP2021/Train-Muc: 1050


In [16]:
evaluate(dev_sentences, dev_labels, hmm_params)


Đang đánh giá mô hình HMM trên tập kiểm thử...


Đang dự đoán: 100%|██████████| 450/450 [22:47<00:00,  3.04s/it]


                       precision    recall  f1-score   support

            B-ADDRESS       0.00      0.00      0.00        30
           B-DATETIME       0.64      0.11      0.19      2305
      B-DATETIME-DATE       0.51      0.08      0.13       688
 B-DATETIME-DATERANGE       0.21      0.04      0.07       316
  B-DATETIME-DURATION       0.54      0.05      0.10       902
       B-DATETIME-SET       0.00      0.00      0.00        73
      B-DATETIME-TIME       0.46      0.15      0.22        89
 B-DATETIME-TIMERANGE       0.03      0.01      0.01       159
              B-EMAIL       0.00      0.00      0.00         1
              B-EVENT       0.23      0.12      0.16       516
          B-EVENT-CUL       0.30      0.09      0.13       129
     B-EVENT-GAMESHOW       0.00      0.00      0.00       154
      B-EVENT-NATURAL       0.96      0.44      0.60        57
        B-EVENT-SPORT       0.19      0.12      0.15        97
           B-LOCATION       0.50      0.04      0.08  

In [39]:
def load_test_data(file_path):
    """
    Đọc dữ liệu từ file tests.txt và chuyển đổi sang định dạng phù hợp,
    chỉ lấy token và bỏ qua POS tag.
    """
    sentences = []
    current_sentence = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                word = line.split()[0]  # Chỉ lấy token đầu tiên (word)
                current_sentence.append(word)
            else:
                sentences.append(current_sentence)
                current_sentence = []
        if current_sentence:  # Xử lý trường hợp câu cuối cùng không có dòng trống
            sentences.append(current_sentence)
    return sentences

In [40]:
def predict_test_data(test_sentences, hmm_params):
    """
    Dự đoán nhãn cho các câu trong tập test và trả về kết quả.
    """
    initial_probs, transition_probs, emission_probs, states = hmm_params
    all_tokens = []  # Danh sách chứa tất cả các token
    all_labels = []  # Danh sách chứa tất cả các nhãn dự đoán

    for sentence in tqdm(test_sentences, desc="Predicting"):
        pred = viterbi(sentence, initial_probs, transition_probs, emission_probs, states)
        all_tokens.extend(sentence)  # Thêm token vào danh sách
        all_labels.extend(pred)  # Thêm nhãn dự đoán vào danh sách

    return all_tokens, all_labels

In [41]:
def load_data_with_groundtruth(file_path):
    """
    Đọc dữ liệu từ file (như tests.txt) và chuyển đổi sang định dạng phù hợp,
    lấy cả token và nhãn NER groundtruth.
    """
    sentences, labels = [], []
    current_sentence, current_labels = [], []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                token, label = line.split()[:2]  # Lấy 2 cột đầu: token và nhãn
                current_sentence.append(token)
                current_labels.append(label)
            else:
                sentences.append(current_sentence)
                labels.append(current_labels)
                current_sentence, current_labels = [], []

        # Xử lý trường hợp câu cuối cùng không có dòng trống
        if current_sentence:
            sentences.append(current_sentence)
            labels.append(current_labels)

    return sentences, labels

In [48]:
# Load groundtruth data and predictions
test_sentences, test_labels = load_data_with_groundtruth(test_file)

# Calculate the size of the test subset (1/3 of the data)
#test_size = len(test_sentences) // 3

# Create subsets for evaluation
#test_sentences_subset = test_sentences[:test_size]
#test_labels_subset = test_labels[:test_size]

# Predict labels for the subset
all_tokens, all_labels = predict_test_data(test_sentences, hmm_params)

# Flatten test_labels for the subset
flat_test_labels = [label for sublist in test_labels for label in sublist]

# In classification_report for the subset
from sklearn.metrics import classification_report
print(classification_report(flat_test_labels, all_labels, zero_division=0))

# Ghi kết quả dự đoán vào file predict.txt for the subset
with open('predict.txt', 'w', encoding='utf-8') as outfile:
    for i, sentence in enumerate(test_sentences):
        for j, word in enumerate(sentence):
            outfile.write(f"{word} {test_labels[i][j]} {all_labels[sum(len(s) for s in test_sentences[:i]) + j]}\n")
        outfile.write("\n")

print("Kết quả dự đoán đã được ghi vào file predict.txt")

Predicting: 100%|██████████| 310/310 [13:05<00:00,  2.53s/it]


                       precision    recall  f1-score   support

            B-ADDRESS       0.13      0.03      0.05       204
           B-DATETIME       0.45      0.10      0.16      1283
      B-DATETIME-DATE       0.60      0.13      0.21      1087
 B-DATETIME-DATERANGE       0.18      0.04      0.06       327
  B-DATETIME-DURATION       0.44      0.07      0.12       716
       B-DATETIME-SET       0.00      0.00      0.00        13
      B-DATETIME-TIME       0.19      0.05      0.09        73
 B-DATETIME-TIMERANGE       0.51      0.22      0.31       165
              B-EMAIL       0.00      0.00      0.00         2
              B-EVENT       0.39      0.06      0.10       756
          B-EVENT-CUL       0.00      0.00      0.00        34
     B-EVENT-GAMESHOW       0.52      0.06      0.11       248
      B-EVENT-NATURAL       0.67      0.08      0.15        24
        B-EVENT-SPORT       0.76      0.10      0.17       491
                 B-IP       0.00      0.00      0.00  

In [53]:
# In bảng xác suất chuyển trạng thái
initial_probs, transition_probs, emission_probs, states = hmm_params
for current_state, next_states in transition_probs.items():
    print(f"Xác suất chuyển trạng thái từ {current_state}:")
    for next_state, prob in next_states.items():
        print(f"  - Đến {next_state}: {prob:.4f}")  # In với 4 chữ số thập phân
    print()

Xác suất chuyển trạng thái từ O:
  - Đến B-PRODUCT: 0.0024
  - Đến O: 0.9280
  - Đến B-PERSONTYPE: 0.0047
  - Đến B-LOCATION-GPE: 0.0087
  - Đến B-MISCELLANEOUS: 0.0008
  - Đến B-PERSON: 0.0123
  - Đến B-PRODUCT-COM: 0.0009
  - Đến B-QUANTITY-NUM: 0.0053
  - Đến B-DATETIME-TIME: 0.0005
  - Đến B-LOCATION-STRUC: 0.0007
  - Đến B-ORGANIZATION: 0.0091
  - Đến B-QUANTITY-PER: 0.0005
  - Đến B-DATETIME-DATE: 0.0015
  - Đến B-DATETIME-TIMERANGE: 0.0004
  - Đến B-DATETIME: 0.0025
  - Đến B-QUANTITY-CUR: 0.0009
  - Đến B-DATETIME-DURATION: 0.0010
  - Đến B-DATETIME-DATERANGE: 0.0003
  - Đến B-QUANTITY-DIM: 0.0003
  - Đến B-QUANTITY: 0.0036
  - Đến B-LOCATION: 0.0064
  - Đến B-DATETIME-SET: 0.0000
  - Đến B-ADDRESS: 0.0001
  - Đến B-QUANTITY-AGE: 0.0003
  - Đến I-QUANTITY-NUM: 0.0010
  - Đến B-EVENT: 0.0006
  - Đến B-EVENT-CUL: 0.0003
  - Đến B-URL: 0.0003
  - Đến B-ORGANIZATION-SPORTS: 0.0024
  - Đến B-EVENT-SPORT: 0.0006
  - Đến B-QUANTITY-ORD: 0.0010
  - Đến B-LOCATION-GEO: 0.0005
  - Đến B-